In [11]:
import sourmash
from sourmash import minhash

In [12]:
ls sigs/

GCF_000005845.2_ASM584v2_genomic.fna.gz.sig
GCF_000006945.1_ASM694v1_genomic.fna.gz.sig
GCF_000783305.1_ASM78330v1_genomic.fna.gz.sig


In [51]:
ss = sourmash.load_one_signature('sigs/GCF_000005845.2_ASM584v2_genomic.fna.gz.sig')
mh1 = ss.minhash

ss = sourmash.load_one_signature('sigs/GCF_000006945.1_ASM694v1_genomic.fna.gz.sig')
mh2 = ss.minhash

In [55]:
print(len(mh1), mh1.scaled)
print(len(mh2), mh2.scaled)

4476 1000
4900 1000


In [52]:
mh1.similarity(mh2)

0.008497364741314403

In [37]:
# full range size == 1/1k
max_hash_1k = minhash._get_max_hash_for_scaled(1000) 

# band size = 1/10k
max_hash_10k = minhash._get_max_hash_for_scaled(10000) 

ivals = []

last_i = 0
for n, i in enumerate(range(max_hash_10k, max_hash_1k + 1, max_hash_10k)):
    ivals.append((last_i, i, mh1.copy_and_clear()))
    print(n, last_i, i, i - last_i)
    last_i = i

0 0 1844674407370955 1844674407370955
1 1844674407370955 3689348814741910 1844674407370955
2 3689348814741910 5534023222112865 1844674407370955
3 5534023222112865 7378697629483820 1844674407370955
4 7378697629483820 9223372036854775 1844674407370955
5 9223372036854775 11068046444225730 1844674407370955
6 11068046444225730 12912720851596685 1844674407370955
7 12912720851596685 14757395258967640 1844674407370955
8 14757395258967640 16602069666338595 1844674407370955
9 16602069666338595 18446744073709550 1844674407370955


In [38]:
for hashval in mh1.hashes:
    for (start, end, band_mh) in ivals:
        if hashval >= start and hashval < end:
            band_mh.add_hash(hashval)
            break

In [48]:
def intersection(mh, ivals):
    result = mh.copy_and_clear()
    for (_, _, band_mh) in ivals:
        result += mh.intersection(band_mh)
    return result

def union(mh, ivals):
    result = mh.copy_and_clear()
    for (_, _, band_mh) in ivals:
        result += mh | band_mh
    return result

In [63]:
r = intersection(mh2, ivals)
r2 = mh2 & mh1
print(len(r), len(r2), r.similarity(r2))
assert r == r2
r.similarity(mh1), mh1.similarity(mh2)

79 79 1.0


(0.017649687220732797, 0.008497364741314403)

In [62]:
r = union(mh2, ivals)
r2 = mh2 | mh1
print(len(r), len(r2), r.similarity(r2))
assert r == r2
#r.similarity(mh1)
r.similarity(mh1), mh1.similarity(mh2)

9297 9297 1.0


(0.4814456276218135, 0.008497364741314403)